<img src="./img/logo_UTN.svg" align="right" width="200" /> 

## Medidas Electrónicas I

# Guia de Ejercicios - 2024
## Trabajo Práctico N°4 - Osciloscopia - Ejercicio N°12
### Tomas A. Albanesi

## Indice de contenidos:
* [1. Consigna](#1)
* [2. Inicializaciones de código](#2)
* [3. Resolución analítica y numérica](#3)

## 1. Consigna <a class="anchor" id="1"></a>

<img src="./img/TP4EJ12_Consigna.png" align="left" width="600" /> 

## 2. Inicializaciones de código <a class="anchor" id="2"></a>

In [3]:
# Importación de librerías
from IPython.display import display, Math, Markdown
import numpy as np
import sympy as sp

# Funciones útiles
def print_latex(unstr):
    if not isinstance(unstr, str):
        raise ValueError("unstr debe ser una cadena.")   
    display(Math(unstr))

def print_subtitle(unstr):
    if not isinstance(unstr, str):
        raise ValueError("unstr debe ser una cadena.") 
    display(Markdown('#### ' + unstr))

def to_latex(unsimbolo):
    if not isinstance(unsimbolo, (sp.Expr, str)):
        raise ValueError("unsimbolo debe ser un símbolo o una cadena.")
    a_str = sp.latex(unsimbolo) if isinstance(unsimbolo, sp.Expr) else unsimbolo  
    return '$' + a_str + '$'

## 3. Resolución analítica y numérica Item A) <a class="anchor" id="3"></a>

En desarrollo ...

## 4. Resolución analítica y numérica Item B) <a class="anchor" id="4"></a>

In [54]:
# Datos del problema

# Número de divisiones de tensión y corriente
DIVISIONES_V = 4 
DIVISIONES_I = 3

# Escalas de canales 1 y 2
FACTOR_CH1 = 1       #1V/div
FACTOR_CH2 = 10e-3   #10mV/div

# Valores de componentes
R = 1

# Punta de prueba
K = 10

# Base de tiempo
FBT = 1e-6

# Cálculo de tensión y corriente
V = DIVISIONES_V * FACTOR_CH1 * K
I = DIVISIONES_I * FACTOR_CH2 / R * K

# Función de medición (Esto tiene incertidumbre)
Z_mod = V/I 

print(f"La impedancia es: {Z_mod}")

La impedancia es: 133.33333333333334


La tensión tiene error por lo siguiente:
- Lectura del valor
- Adquisición a traves del canal
- Punta de prueba

La corriente tiene error por lo siguiente:
- Lectura del valor
- Adquisición a traves del canal
- Punta de prueba
- Resistencia (en este caso no presenta error)

<b>El camino relativo lo que haría es: Sumar cuadraticamente las incertidumbres relativas de cada una de las fuentes de incertidumbre.</b>

<b>¡ESTE CAMINO RELATIVO FUNCIONA SIEMPRE PARA FUNCIONES DE MEDICIONES QUE SEAN MULTIPLICACIONES O DIVISIONES!</b>

Entonces, ya que en este caso no tenemos incertidumbre tipo A y utilizamos el camino relativo:

$$
\begin{equation}
u_{c}(V) = \sqrt{u²_{div{_\%}} + u²_{fch1{_\%}} + u²_{k{_\%}}}
\end{equation}
$$

Por lo tanto, en primer lugar, planteamos el error de lectura:

Si observamos el dibujo, el error que podemos cometer es de una división menor, que son $0,2$ divisiones de la mayor.

Entonces planteamos la siguiente regla de tres simple:

- Si $4$ divisiones $=$ $100\%$
- Entonces $0.2$ divisiones $=$ $?$

In [18]:
u_divV_p = (0.2 / DIVISIONES_V) / np.sqrt(3)

Los demas valores directamente son porcentuales:

In [19]:
u_fch1_p = 0.03/np.sqrt(3)
u_k_p = 0.02/np.sqrt(3)

In [25]:
uc_V_rel = np.sqrt(u_divV_p**2 + u_fch1_p**2 + u_k_p**2)

print(f"La incertidumbre combinada relativa de la tensión es: {uc_V_rel}")

La incertidumbre combinada relativa de la tensión es: 0.035590260840104374


Hacemos algo parecido con la corriente.

In [21]:
u_divI_p = (0.2 / DIVISIONES_I) / np.sqrt(3)
u_fch2_p = 0.03/np.sqrt(3)
u_k_p = 0.02/np.sqrt(3)
u_r = 0.01/np.sqrt(3)

In [30]:
uc_I_rel = np.sqrt(u_divI_p**2 + u_fch2_p**2 + u_k_p**2 + u_r**2)

print(f"La incertidumbre combinada relativa de la corriente es: {uc_I_rel}")

La incertidumbre combinada relativa de la corriente es: 0.044137831257869346


Ahora podemos calcular la incertidumbre combinada relativa de la impedancia.

In [31]:
uc_Z_rel = np.sqrt(uc_V_rel**2 + uc_I_rel**2)

print(f"La incertidumbre combinada relativa de la impedancia es: {uc_Z_rel}")

La incertidumbre combinada relativa de la impedancia es: 0.056699336987435886


In [33]:
print(f"|Z| = {int(Z_mod)}, uc(Z) = {int(uc_Z_rel * Z * 2)}")

Z = 133, uc(Z) = 15


Lo que queremos hallar son los componentes.
Ya tenemos el módulo de la impedancia $Z$.
Nos falta calcular la fase.

$\phi_Z = \phi_V - \phi_I$

Calculamos por medio del gráfico, las diferencias de fases.

Para hacerlo, contamos las divisiones entre los cruces por cero.

Podemos contar que hay 8 subdivisiones que equivale a 1,6 divisiones.

Ahora podemos utilizar la siguiente fórmula para calcular el desfasaje en grados:

$$
\begin{equation}
\phi = \frac{2 \cdot \pi \cdot \tau}{T}
\end{equation}
$$

O directamente aplicar la siguiente regla de tres simple:

- Si 10 divisiones = 360°
- Entonces -1,6 divisiones = ? (-57,6)

El signo menos es debido a que la corriente adelanta a la tensión.

In [41]:
T = 10
dT = -1.6

Z_phi = (2*np.pi*dT)/T
Z_phi

-1.0053096491487339

In [46]:
# Fuente de error
err_lectura_dT = (0.2/1.6) / np.sqrt(3)
err_lectura_horizontal = 0.03 / np.sqrt(3)
err_lectura_T = (0.2/10) / np.sqrt(3)

# Incertidumbre
uc_dT = np.sqrt(err_lectura_dT**2 + err_lectura_horizontal**2)
uc_T = np.sqrt(err_lectura_T**2 + err_lectura_horizontal**2)

# Incertidumbre combinada de la fase de la impedancia
uc_Z_phi = np.sqrt(uc_dT**2 + uc_T**2)
uc_Z_phi

np.float64(0.07708220719898119)

In [56]:
Xc = Z_mod * np.sin(Z_phi)
Xc

np.float64(-112.57705673360202)

In [57]:
C = 1 / (np.abs(Xc) * 2 * np.pi * (1/(T * FBT)))
C

np.float64(1.4137422642742686e-08)